In [1]:
import os
import numpy as np
import torch
import random
import parmed
import nglview as ng
import MDAnalysis as md
from MDAnalysis.analysis import dihedrals
import matplotlib.pyplot as plt

import mymd

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
data_path = '../../data/md_from_scratch/1ytc'
csv_path = '../../data/md_from_scratch/1ytc/rslt_mymd'
dcd_path = '../../data/md_from_scratch/1ytc/rslt_mymd'

old_pdb_file = os.path.join(data_path, '1ytc_fixed.pdb')
psf_file = os.path.join(data_path, '1ytc_fixed_mymd.psf')
prmtop_file = os.path.join(data_path, '1ytc_fixed_mymd.prmtop')
pdb_file = os.path.join(data_path, '1ytc_fixed_mymd.pdb')
coor_file = None
xsc_file = None

cutoff = None
T = 300.15
dt_fs = 2
dcd_interval = 100
csv_interval = 50
# steps = 50000
steps = 500
# min_energy_max_iter = 500
min_energy_max_iter = 10
box_size = 100  # can not be too small, e.g. 20 do not work, 100 is good
device = 'cpu'
precision = torch.float
use_centered = True

In [4]:
# mymd.gen_files_from_pdb(old_pdb_file=old_pdb_file,
#                         psf_file=psf_file,
#                         prmtop_file=prmtop_file,
#                         new_pdb_file=pdb_file,
#                         other_ff_file='amber14/tip3pfb.xml')

In [5]:
mol = mymd.get_molecule(prmtop_file=prmtop_file, pdb_file=pdb_file, coor_file=coor_file, xsc_file=xsc_file)

print('---coords---')
print(mol.coords[:2])
print('-----box-----')
print(mol.box)
print('---charge---')
print(mol.charge[:3])
print('---masses---')
print(mol.masses[:3])
print('----bonds----')
print(mol.bonds[:4])
print('---bonds.shape---')
print(mol.bonds.shape)

2023-05-11 17:53:40,315 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


---coords---
[[[-1.787]
  [ 4.614]
  [-1.979]]

 [[-2.232]
  [ 4.031]
  [-2.935]]]
-----box-----
[[0.]
 [0.]
 [0.]]
---charge---
[0.1414 0.1997 0.1997]
---masses---
[14.00672   1.007947  1.007947]
----bonds----
[[ 4  5]
 [ 8  9]
 [ 8 10]
 [ 8 11]]
---bonds.shape---
(2000, 2)


In [6]:
try:
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=False)
except:
    print('False causes error, use True.')
    ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=True)

print(ff.mol.atomtype[:10])
print(ff.mol.charge[:10])
print(set(ff.mol.atomtype))
print(set(ff.params.atom_types))

False causes error, use True.
['N1' 'H1' 'H1' 'H1' 'C1' 'H2' 'C2' 'O1' 'C1' 'H3']
[ 0.1414  0.1997  0.1997  0.1997  0.0962  0.0889  0.6163 -0.5722 -0.0597
  0.03  ]
{'O1', 'H3', 'H6', 'H7', 'N1', 'O3', 'H1', 'H4', 'H2', 'C2', 'H5', 'C1', 'H8', 'S1', 'O2'}
{'O1', 'H3', 'H6', 'H7', 'N1', 'O3', 'H1', 'H4', 'H2', 'C2', 'H5', 'C1', 'H8', 'S1', 'O2'}


In [7]:
system = mymd.System(mol, ff, cutoff=cutoff)
system.set_periodic_box_manual(np.array([box_size, box_size, box_size]).reshape(3, 1))

print(system.bond_params[:5])
print(system.improper_params[0]['params'][:5])
print(system.uni_atom_types == np.unique(mol.atomtype))
print(system.uni_atom_types[system.mapped_atom_types[:6]] == mol.atomtype[:6])
print(system.box, mol.box)
print(system.cutoff)
print(system.external)
print(np.array(system.get_exclusions(types=['bonds']))[:3])
print(np.all(np.vstack((np.array(system.bonds), np.array(system.angles[:, [0, 2]]))) == np.array(system.get_exclusions(types=['bonds', 'angles']))))

tensor([[434.0000,   1.0100],
        [434.0000,   1.0100],
        [434.0000,   1.0100],
        [367.0000,   1.4710],
        [340.0000,   1.0900]])
tensor([[10.5000,  3.1416,  2.0000],
        [ 1.1000,  3.1416,  2.0000],
        [10.5000,  3.1416,  2.0000],
        [ 1.1000,  3.1416,  2.0000],
        [10.5000,  3.1416,  2.0000]])
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True]
[ True  True  True  True  True  True]
tensor([100, 100, 100]) [[0.]
 [0.]
 [0.]]
None
None
[[0 1]
 [0 2]
 [0 3]]
True


In [8]:
integrator = mymd.VelocityVerletIntegrator(dt_fs)
print(integrator.dt, integrator.dt_fs)

0.04090965899549196 2


In [9]:
simulation = mymd.Simulation(mol, system, integrator, device, precision, use_centered=use_centered)
simulation.set_positions(mol.coords)
simulation.set_velocities_to_temperature(T=T)
simulation.update_potentials_and_forces()

csv_reporter = mymd.CSVReporter(csv_path, csv_interval)
simulation.add_reporter(csv_reporter)

print(torch.abs(simulation.vel).mean())
print(simulation.forces.shape)
print(simulation.forces[:5])
print(simulation.forces[-3:])
print(simulation.potentials)
print(simulation.potentials_sum)

Remove the original csv file.
tensor(0.4110)
torch.Size([2098, 3])
tensor([[-315.1233, -171.3560,  -36.8644],
        [  86.8119,   87.0813,  119.6202],
        [  64.7976,  127.1715,  -85.7573],
        [ 160.9893,  -58.1740,   -7.8573],
        [   8.5384,  -19.2624,   51.8338]])
tensor([[ 141.9062,  105.8601,  141.3273],
        [-158.6487,  -23.4577,   35.6042],
        [  23.0708,  -87.7474, -160.4683]])
{'bonds': 9543.9296875, 'angles': 4640.02392578125, 'dihedrals': 2241.162109375, 'impropers': 68.10431671142578, 'lj': 237.96629333496094, 'electrostatics': -455.9336853027344}
16275.252647399902


In [10]:
print(simulation.pos[:5])
simulation.minimize_energy(min_energy_max_iter)
print(simulation.pos[:5])

print(simulation.forces[:5])
print(simulation.potentials)
print(simulation.potentials_sum)

tensor([[-1.7870,  4.6140, -1.9790],
        [-2.2320,  4.0310, -2.9350],
        [-2.0720,  3.7220, -1.2290],
        [-2.7530,  5.3160, -1.8730],
        [-0.3750,  4.9820, -2.0950]])
num_fun_eval	E_pot    	norm_F_max	rms      	max_abs_f_1d
     0      	16275.252647	393.896279	77.616347	392.359009
     1      	11137.180679	239.741846	52.684587	226.678299
     2      	5421.629616	105.942107	17.455346	94.465836
     3      	4385.033199	94.869501	14.870279	74.645523
     4      	3065.223221	179.394562	18.733471	144.910400
     5      	2479.131329	115.418825	15.637045	104.355934
     6      	2146.091038	62.547415	8.635792	52.778023
     7      	1747.105171	67.256120	6.649239	56.488068
     8      	1484.253304	65.538343	7.175730	53.795944
     9      	1248.849564	254.796145	13.149290	201.917999
     10     	1026.681389	108.385188	6.548044	84.860443
tensor([[-1.8466,  4.4624, -2.0687],
        [-2.1816,  4.0449, -2.9359],
        [-1.9824,  3.8356, -1.2841],
        [-2.3636,  5.3134, -1.8

In [11]:
simulation.step(steps, dcd_path, dcd_interval)

100%|██████████| 500/500 [10:11<00:00,  1.22s/it]


In [12]:
u = md.Universe(psf_file, os.path.join(dcd_path, 'traj.dcd'))
ng.show_mdanalysis(u, gui=True)

2023-05-11 18:04:48,557 - MDAnalysis.topology.PSF - WARNING - Guessing that this is actually a NAMD-type PSF file... continuing with fingers crossed!


NGLWidget(max_frame=5)